In [1]:
# инстоллы найдены путём проб и ошибок,
# без них всё ломается
# !pip install transformers
# !pip install -U easynmt
# !pip install datasets

# !pip install sacremoses
# !pip install sentencepiece

# Obtain RACE dataset

In [2]:
from datasets import load_dataset, load_dataset_builder

dataset_builder = load_dataset_builder("race", "all")
dataset_train = load_dataset("race", "all", split="train").select(range(4000))

Reusing dataset race (C:\Users\User\.cache\huggingface\datasets\race\all\0.1.0\5839ff74a429622f5f20cca69c5fcf0e87ac6d5fd2777c42b948000684829f7b)


# Translate RACE with EasyNMT

In [3]:
from easynmt import EasyNMT

In [4]:
model = EasyNMT('opus-mt')

In [5]:
def translate(example, batch_size=16):
  return {
      "article_ru": model.translate(
          example["article"],
          target_lang="ru",
          source_lang="en",
          batch_size=batch_size
      ),
      "options_ru": [model.translate(
          option,
          target_lang="ru",
          source_lang="en",
          batch_size=batch_size
      ) for option in example["options"]],
      "question_ru": model.translate(
          example["question"],
          target_lang="ru",
          source_lang="en",
          batch_size=batch_size
      )
  }

In [6]:
%%time
batch_size = 16
dataset_train_ru = dataset_train.map(
    lambda x: translate(x, batch_size),
    batched=True,
    batch_size=batch_size
)

Parameter 'function'=<function <lambda> at 0x000001C547A63160> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|██████████| 250/250 [1:43:58<00:00, 24.95s/ba]

CPU times: total: 3h 50min 3s
Wall time: 1h 43min 58s


In [7]:
dataset_train_ru.to_csv(
    "race_ru_train_0_4000.csv"
)

Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  1.49ba/s]


22972999